In [1]:
from Okezone import Okezone
from datetime import timedelta, date, datetime
from IPython.display import clear_output

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [2]:
obj = Okezone()

In [3]:
end_date = date(2018, 8, 12)
start_date = date(2018, 3, 27)
init_page = 18
init_offset = 255

In [4]:
for single_date in daterange(start_date, end_date):
    clear_output()
    print(single_date.strftime("%Y-%m-%d"))
    obj.getAllBerita([], init_page, init_offset, single_date.strftime("%Y/%m/%d"))
    init_page = 1
    init_offset = 0

2018-05-03
page  1
https://index.okezone.com/bydate/index/2018/05/03/0/
memasukkan berita id  1894238
Insert berita  Jelang Tahun Politik, Jangan Jadikan Masjid Tempat Kampanye
masuk
memasukkan berita id  1894252
Insert berita  Luis Milla: <i>Good Job</i> Timnas Indonesia U-23!
masuk
memasukkan berita id  1894249
Insert berita  Setahun Digulirkan, Program Layad Rawat Sabet Penghargaan IHFC Innovation Award 2018
masuk
memasukkan berita id  1894222
Insert berita  PDIP Pertimbangkan JK Jadi Cawapres Jokowi bila MK Kabulkan Uji Materi
masuk
memasukkan berita id  1894194
Insert berita  Kehadiran Zarco Diharapkan Bisa Membawa KTM Tampil Lebih Baik di MotoGP 2019
masuk
memasukkan berita id  1894175
Insert berita  Kompak, Hamish Daud dan Raisa Sama-sama Pecinta Kopi
masuk
memasukkan berita id  1894231
Insert berita  Wenger Tak Pikirkan Hasil Buruk Tandang Arsenal Jelang Lawan Atletico Madrid
masuk
memasukkan berita id  1894128
Insert berita  Pujian Bek Persib untuk Madura United
masuk
memasukk

masuk
memasukkan berita id  1894109
Insert berita  Rossi Yakin Yamaha YZR-M1 Akan Tunjukkan Potensinya di Sirkuit Jerez
masuk
memasukkan berita id  1894180
Insert berita  Billy Syahputra Bangkrut, Ini Kata Hilda Vitria
masuk
memasukkan berita id  1894159
Insert berita  Bamsoet Khawatir Jutaan Warga Tak Bisa Nyoblos di Pilkada 2018
masuk
memasukkan berita id  1894178
Insert berita  Cerita Perjuangan 2 Pejuang Air Demi Menjaga Kelestarian Lingkungan
masuk
memasukkan berita id  1894177
Insert berita  Antisipasi Kelangkaan BBM dan Elpiji, Pertamina Bentuk Tim Satgas Pemantauan Distribusi
masuk
memasukkan berita id  1894173
Insert berita  Sosok Pacar Putri Jackie Chan Kerap Tampilkan Foto Seksi di Instagram
masuk
memasukkan berita id  1894169
Insert berita  IFAD: Banyak Negara Belajar dari Indonesia dalam Hal Membangun Desa
masuk
page  6
https://index.okezone.com/bydate/index/2018/05/03/75/
memasukkan berita id  1894168
Insert berita  Kylie Jenner Merasa Bangga Menjadi Seorang Ibu
masuk


KeyboardInterrupt: 